## Task 1 using Plotly

In [1]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import VBox, HBox, FloatSlider, Checkbox, Button, Dropdown, Output
from scipy import signal

### Function for creating harmonics with noise

In [2]:
def harmonic_with_noise(amplitude, frequency, phase, noise_mean, noise_covariance, show_noise, cutoff_frequency=None):
    clean_signal = amplitude * np.sin(2 * np.pi * frequency * time + phase)
    noise = np.random.normal(noise_mean, np.sqrt(noise_covariance), size=time.shape) if show_noise else 0
    noisy_signal = clean_signal + noise
    return clean_signal, noisy_signal

### Function for resetting parameters

In [3]:
def reset_parameters(button):
    amplitude_slider.value = initial_params['amplitude']
    frequency_slider.value = initial_params['frequency']
    phase_slider.value = initial_params['phase']
    noise_mean_slider.value = initial_params['noise_mean']
    noise_covariance_slider.value = initial_params['noise_covariance']
    cutoff_frequency_slider.value = initial_params['cutoff_frequency']
    show_noise_checkbox.value = initial_params['show_noise']
    signal_color_selection.value = "Blue"
    noise_color_selection.value = "Orange"
    update_graph()

### Updating the graph when changing the values ​​of the sliders

In [4]:
def update_graph(change=None):
    global previous_noise_mean, previous_noise_covariance, current_noise

    # Генерация сигналов
    amplitude = amplitude_slider.value
    frequency = frequency_slider.value
    phase = phase_slider.value
    noise_mean = noise_mean_slider.value
    noise_covariance = noise_covariance_slider.value
    show_noise = show_noise_checkbox.value

    if noise_mean != previous_noise_mean or noise_covariance != previous_noise_covariance:
        current_noise = np.random.normal(noise_mean, np.sqrt(noise_covariance), size=time.shape)
        previous_noise_mean = noise_mean
        previous_noise_covariance = noise_covariance
    
    clean_signal = amplitude * np.sin(2 * np.pi * frequency * time + phase)
    noisy_signal = clean_signal + (current_noise if current_noise is not None else 0)
    
    filtered_signal = filter_signal(time, noisy_signal, cutoff_frequency_slider.value)

    signal_color = signal_color_selection.value.lower()
    noise_color = noise_color_selection.value.lower()

    with fig_line.batch_update():
        fig_line.data[0].y = noisy_signal
        fig_line.data[0].line.color = noise_color
        fig_line.data[1].y = clean_signal
        fig_line.data[1].line.color = signal_color
        fig_line.data[2].y = filtered_signal
        fig_line.data[2].line.color = 'red'  # Красный цвет для фильтрованного сигнала

    with fig_dotted.batch_update():
        fig_dotted.data[0].y = noisy_signal
        fig_dotted.data[0].marker.color = noise_color
        fig_dotted.data[1].y = clean_signal
        fig_dotted.data[1].marker.color = signal_color
        fig_dotted.data[2].y = filtered_signal
        fig_dotted.data[2].marker.color = 'red'  # Красный цвет для фильтрованного сигнала

### Signal filtering function

In [5]:
def filter_signal(t, y_noisy, cutoff_frequency):
    fs = 100  # Частота дискретизації
    nyquist = 0.5 * fs  # Частота Найквіста
    normal_cutoff = cutoff_frequency / nyquist  # Нормалізована частота зрізу
    b, a = signal.butter(4, normal_cutoff, btype='low')  # 4-й порядок фільтра
    y_filtered = signal.filtfilt(b, a, y_noisy)  # Фільтрація сигналу
    return y_filtered

### Creating an application

In [6]:
# Початкові параметри
initial_params = {
    "amplitude": 1.0,
    "frequency": 0.5,
    "phase": 0.0,
    "noise_mean": 0.0,
    "noise_covariance": 0.1,
    "show_noise": True,
    "cutoff_frequency": 5.0,
}

time = np.linspace(0, 10, 1000)

previous_noise_mean = initial_params["noise_mean"]
previous_noise_covariance = initial_params["noise_covariance"]
current_noise = np.random.normal(previous_noise_mean, np.sqrt(previous_noise_covariance), size=time.shape)

# Генерація початкових сигналів
clean_signal, noisy_signal = harmonic_with_noise(**initial_params)
filtered_signal = filter_signal(time, noisy_signal, initial_params['cutoff_frequency'])

# Графіки
fig_line = go.FigureWidget()
fig_line.add_scatter(x=time, y=noisy_signal, mode='lines', name='Noisy Signal', line=dict(color='orange'))
fig_line.add_scatter(x=time, y=clean_signal, mode='lines', name='Clean Signal', line=dict(color='blue'))
fig_line.add_scatter(x=time, y=filtered_signal, mode='lines', name='Filtered Signal', line=dict(color='green'))
fig_line.update_layout(title="Line Plot", xaxis_title="Time", yaxis_title="Amplitude")

fig_dotted = go.FigureWidget()
fig_dotted.add_scatter(x=time, y=noisy_signal, mode='markers', name='Noisy Signal', marker=dict(color='orange'))
fig_dotted.add_scatter(x=time, y=clean_signal, mode='markers', name='Clean Signal', marker=dict(color='blue'))
fig_dotted.add_scatter(x=time, y=filtered_signal, mode='markers', name='Filtered Signal', marker=dict(color='green'))
fig_dotted.update_layout(title="Dotted Plot", xaxis_title="Time", yaxis_title="Amplitude")

# Віджети
amplitude_slider = FloatSlider(value=initial_params['amplitude'], min=0.1, max=2.0, step=0.1, description='Amplitude')
frequency_slider = FloatSlider(value=initial_params['frequency'], min=0.1, max=2.0, step=0.1, description='Frequency')
phase_slider = FloatSlider(value=initial_params['phase'], min=-np.pi, max=np.pi, step=0.1, description='Phase')
noise_mean_slider = FloatSlider(value=initial_params['noise_mean'], min=-1.0, max=1.0, step=0.1, description='Noise Mean')
noise_covariance_slider = FloatSlider(value=initial_params['noise_covariance'], min=0.0, max=1.0, step=0.1, description='Noise Cov')
cutoff_frequency_slider = FloatSlider(value=initial_params['cutoff_frequency'], min=0.0, max=10.0, step=0.1, description='Cutoff Frequency')
show_noise_checkbox = Checkbox(value=initial_params['show_noise'], description='Show Noise')
reset_button = Button(description="Reset", button_style="info")
signal_color_selection = Dropdown(options=["Orange", "Blue", "Green", "Red"], value="Blue", description="Signal Color:")
noise_color_selection = Dropdown(options=["Orange", "Blue", "Green", "Red"], value="Orange", description="Noise Color:")

# Прив'язка подій
amplitude_slider.observe(update_graph, names='value')
frequency_slider.observe(update_graph, names='value')
phase_slider.observe(update_graph, names='value')
noise_mean_slider.observe(update_graph, names='value')
noise_covariance_slider.observe(update_graph, names='value')
cutoff_frequency_slider.observe(update_graph, names='value')
show_noise_checkbox.observe(update_graph, names='value')
signal_color_selection.observe(update_graph, names='value')
noise_color_selection.observe(update_graph, names='value')
reset_button.on_click(reset_parameters)

# Відображення
ui = VBox([
    HBox([amplitude_slider, frequency_slider, phase_slider]),
    HBox([noise_mean_slider, noise_covariance_slider, show_noise_checkbox]),
    HBox([signal_color_selection, noise_color_selection, reset_button]),
])

VBox([ui, fig_line, fig_dotted])